In [ ]:
from mastodon import Mastodon

API = Mastodon(
    client_id='b-1LjDLQvdfa1Kd_uVk4EStyVGWCe_lqipdwKG68LGw',
    client_secret='jkWM5f58nofXVQP9-x0sf6yzuB3WfePTiIaHLe97kwM',
    access_token='zuG68yri_Fp4RxFro2QzRK4RRAp1bcmFwyRc5Rjb9Sw',
    api_base_url='https://mastodon.social'  
)

me = API.account_verify_credentials()

Account([('id', 115469357557448283), ('username', 'vuTester'), ('acct', 'vuTester'), ('display_name', 'bendover'), ('discoverable', True), ('group', False), ('locked', False), ('created_at', datetime.datetime(2025, 10, 31, 0, 0, tzinfo=tzutc())), ('following_count', 0), ('followers_count', 0), ('statuses_count', 0), ('note', '<p>bendover</p>'), ('url', 'https://mastodon.social/@vuTester'), ('uri', 'https://mastodon.social/ap/users/115469357557448283'), ('avatar', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('avatar_static', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header_static', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('moved', None), ('suspended', None), ('limited', None), ('bot', 

In [ ]:
import requests
import time
from datetime import datetime, timedelta, timezone

# -----------------------------------
# 1. Instances (no API keys needed)
# -----------------------------------
INSTANCES = [
    "mastodon.social",
    "sigmoid.social",
    "hachyderm.io",
    "fediscience.org",  
    "universeodon.com",
    "techhub.social",
    "mstdn.social",
]

# -----------------------------------
# 2. Events (example: fill out more)
#    Use hashtags without the '#'
# -----------------------------------

events = [
    {
        "name": "Stable Diffusion public release",
        "date": datetime(2022, 8, 22, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "Stability AI releases Stable Diffusion as a publicly available latent diffusion image model, catalyzing the open-source generative image ecosystem.",
        "hashtags": [
            "stable diffusion", "stablediffusion", "stability ai",
            "dreamstudio", "sd 1.4", "sd 1.5"
        ],
        "window_days": 14,
    },
    {
        "name": "DALL·E 2 research preview",
        "date": datetime(2022, 4, 6, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "OpenAI unveils DALL·E 2 as a research preview, demonstrating high-quality text-to-image generation and popularizing the modern wave of AI art models.",
        "hashtags": [
            "dall-e 2", "dalle 2", "dall e 2", "openai image",
            "text-to-image", "ai art"
        ],
        "window_days": 14,
    },
    {
        "name": "Midjourney v4 beta",
        "date": datetime(2022, 11, 10, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "Midjourney releases version 4 in beta, substantially improving image fidelity and becoming a dominant creative GenAI tool in online art communities.",
        "hashtags": [
            "midjourney", "midjourney v4", "mj v4",
            "aiart", "synthography"
        ],
        "window_days": 14,
    },
    {
        "name": "ChatGPT launch (GPT-3.5-based)",
        "date": datetime(2022, 11, 30, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "OpenAI releases ChatGPT as a research preview, triggering mass public adoption of conversational generative AI and a global AI boom.",
        "hashtags": [
            "chatgpt", "chat gpt", "gpt-3.5", "gpt3.5",
            "openai chatbot"
        ],
        "window_days": 14,
    },
    {
        "name": "GPT-4 release",
        "date": datetime(2023, 3, 14, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "OpenAI announces GPT-4, a more capable multimodal large language model, used in ChatGPT and integrated into products like Microsoft Copilot.",
        "hashtags": [
            "gpt-4", "gpt4", "gpt 4",
            "gpt-4 api", "gpt-4 release", "gpt4 turbo"
        ],
        "window_days": 14,
    },
    {
        "name": "Llama 2 open-source release",
        "date": datetime(2023, 7, 18, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "Meta and Microsoft release Llama 2 with a permissive license, greatly accelerating the open-source GenAI ecosystem for text generation.",
        "hashtags": [
            "llama 2", "llama2", "meta llama",
            "open source llm", "meta ai", "llm"
        ],
        "window_days": 14,
    },
    {
        "name": "OpenAI Dev Day (GPT-4 Turbo, GPTs, Assistants API)",
        "date": datetime(2023, 11, 6, tzinfo=timezone.utc),
        "category": "breakthrough",
        "description": "At its first Dev Day, OpenAI announces GPT-4 Turbo, multimodal upgrades, GPTs, and the Assistants API, marking a platform shift for GenAI applications.",
        "hashtags": [
            "openai dev day", "devday", "gpt-4 turbo",
            "assistants api", "custom gpts", "openai event"
        ],
        "window_days": 14,
    },
    {
        "name": "Sam Altman removal and reinstatement at OpenAI",
        "date": datetime(2023, 11, 17, tzinfo=timezone.utc),
        "category": "controversy",
        "description": "OpenAI's board fires CEO Sam Altman on November 17, 2023, prompting a global backlash and his reinstatement within days, sparking debate about AI governance and safety.",
        "hashtags": [
            "sam altman", "openai board", "altman firing",
            "openai governance", "openai crisis"
        ],
        "window_days": 14,
    },
    {
        "name": "EU AI Act provisional agreement",
        "date": datetime(2023, 12, 9, tzinfo=timezone.utc),
        "category": "regulation",
        "description": "EU institutions reach a provisional deal on the AI Act, introducing risk-based rules and specific obligations for general-purpose and foundation models, including GenAI systems.",
        "hashtags": [
            "eu ai act", "ai act", "european ai law",
            "foundation models", "gpaI", "genai regulation"
        ],
        "window_days": 14,
    },
    {
        "name": "Getty Images v. Stability AI (copyright dispute over training data)",
        "date": datetime(2023, 1, 16, tzinfo=timezone.utc),
        "category": "controversy",
        "description": "Getty Images sues Stability AI over alleged unauthorized use of copyrighted images to train Stable Diffusion, becoming a landmark case on GenAI training data and copyright.",
        "hashtags": [
            "getty images", "stability ai lawsuit",
            "stable diffusion lawsuit", "ai copyright",
            "training data scraping"
        ],
        "window_days": 14,
    },
]

HEADERS = {
    "User-Agent": "AcademicResearchBot/1.0 (contact: your.email@example.com)"
}

# -----------------------------------
# 3. Helpers
# -----------------------------------
def to_datetime(dt):
    if isinstance(dt, datetime):
        return dt
    if dt.endswith("Z"):
        dt = dt[:-1] + "+00:00"
    return datetime.fromisoformat(dt)

def safe_get(url, params=None, max_retries=3, timeout=30):
    for attempt in range(max_retries):
        try:
            r = requests.get(url, params=params, headers=HEADERS, timeout=timeout)
            r.raise_for_status()
            return r
        except requests.exceptions.RequestException as e:
            print(f"  Request error ({attempt+1}/{max_retries}) on {url}: {e}")
            time.sleep(2 * (attempt + 1))
    return None

def count_tag_in_window(instance, tag, start_dt, end_dt, max_pages=200, limit=40, sleep=1.0, debug=False):
    """
    Count how many statuses on `instance` tagged with `tag`
    fall within [start_dt, end_dt).
    Uses /api/v1/timelines/tag/{tag}?local=true (no auth).
    """
    base = f"https://{instance}/api/v1/timelines/tag/{tag}"
    max_id = None
    total = 0

    if debug:
        print(f"[DEBUG]  Instance={instance}, tag={tag}, window={start_dt} -> {end_dt}")

    for page in range(max_pages):
        params = {"local": "true", "limit": limit}
        if max_id is not None:
            params["max_id"] = max_id

        r = safe_get(base, params=params)
        if r is None:
            if debug:
                print("  Giving up on this tag/instance for now.")
            break

        try:
            batch = r.json()
        except ValueError:
            if debug:
                print("  Failed to parse JSON, stopping.")
            break

        if not batch:
            if debug:
                print("  Empty batch, stopping.")
            break

        oldest_in_batch = None
        for s in batch:
            created_at = to_datetime(s["created_at"])
            if oldest_in_batch is None or created_at < oldest_in_batch:
                oldest_in_batch = created_at

            if start_dt <= created_at < end_dt:
                total += 1

        if debug:
            print(f"[DEBUG]  Page {page}, got {len(batch)} statuses, oldest={oldest_in_batch}, total={total}")

        max_id = min(int(s["id"]) for s in batch)

        # If everything in this batch is already older than the window, we can stop
        if oldest_in_batch is not None and oldest_in_batch < start_dt:
            if debug:
                print("[DEBUG]  Oldest status older than start_dt, stopping pagination.")
            break

        time.sleep(sleep)

    return total

# -----------------------------------
# 4. Main: loop over events & instances
# -----------------------------------
results = []

for event in events:
    name = event["name"]
    date = event["date"]
    tags = event["hashtags"]
    window_days = event["window_days"]

    before_start = date - timedelta(days=window_days)
    before_end = date
    after_start = date
    after_end = date + timedelta(days=window_days)

    print(f"\n==================== Event: {name} ({date.date()}) ====================")
    event_result = {"name": name, "instances": {}}

    for inst in INSTANCES:
        print(f"\n--- Instance: {inst} ---")
        inst_result = {"before": {}, "after": {}}

        for tag in tags:
            print(f"Tag: #{tag}")

            before_count = count_tag_in_window(
                inst, tag, before_start, before_end, debug=False
            )
            print(f"  Before: {before_count}")

            after_count = count_tag_in_window(
                inst, tag, after_start, after_end, debug=False
            )
            print(f"  After:  {after_count}")

            inst_result["before"][tag] = before_count
            inst_result["after"][tag] = after_count

        event_result["instances"][inst] = inst_result

    results.append(event_result)

# `results` now holds a nested dict structure with counts per event/instance/tag


==================== Event: Stable Diffusion public release (2022-08-22) ====================

--- Instance: mastodon.social ---
Tag: #stable diffusion
  Before: 0


In [ ]:
import requests
import time
import re
from collections import Counter
from datetime import datetime, timezone
from bs4 import BeautifulSoup

instances = [
    "sigmoid.social",
    "mastodon.ai",
    "ai.community",
    "social.ai",
    "mstdn.art",
    "creators.social",
    "glitch.social",
    "fediscience.org",
    "datasci.social",
    "scicomm.xyz",
    "mastodon.social",
    "techhub.social",
]

START_DATE = datetime(2022, 1, 1, tzinfo=timezone.utc)

stopwords = {
    "the", "and", "for", "that", "this", "you", "with", "are", "was", "have",
    "but", "not", "from", "your", "just", "like", "they", "their", "about",
    "what", "when", "where", "there", "them", "out", "get", "can", "all",
    "one", "will", "into", "more", "also", "some", "has", "had", "our",
    "any", "who", "why", "how", "its", "she", "him", "her", "then", "than",
    "too", "very", "did", "does", "didn", "don", "could", "would", "should",
}

HEADERS = {
    "User-Agent": "AcademicResearchBot/1.0 (contact: your.email@example.com)"
}

def parse_dt(dt_str):
    if isinstance(dt_str, datetime):
        return dt_str
    if dt_str.endswith("Z"):
        dt_str = dt_str[:-1] + "+00:00"
    return datetime.fromisoformat(dt_str)

def safe_get(url, params=None, max_retries=3, timeout=30):
    for attempt in range(max_retries):
        try:
            r = requests.get(url, params=params, headers=HEADERS, timeout=timeout)
            r.raise_for_status()
            return r
        except requests.exceptions.ConnectionError as e:
            print(f"  Connection error ({attempt+1}/{max_retries}): {e}")
        except requests.exceptions.ReadTimeout as e:
            print(f"  Timeout ({attempt+1}/{max_retries}): {e}")
        except requests.exceptions.RequestException as e:
            print(f"  Request failed: {e}")
            break
        time.sleep(2 * (attempt + 1))
    return None

def fetch_instance_statuses(instance, start_date, max_pages=300, limit=40, sleep=3.0):
    base = f"https://{instance}/api/v1/timelines/public"
    max_id = None
    statuses = []

    for _ in range(max_pages):
        params = {"local": "true", "limit": limit}
        if max_id is not None:
            params["max_id"] = max_id

        r = safe_get(base, params=params)
        if r is None:
            print("  Giving up on this instance for now.")
            break

        try:
            batch = r.json()
        except ValueError:
            print("  Failed to parse JSON, stopping.")
            break

        if not batch:
            break

        stop = False
        for s in batch:
            created_at = parse_dt(s["created_at"])
            if created_at < start_date:
                stop = True
                break
            statuses.append(s)

        max_id = min(int(s["id"]) for s in batch)

        if stop:
            break

        time.sleep(sleep)

    return statuses

def extract_words(text):
    text = text.lower()
    text = re.sub(r"https?://\S+", " ", text)
    return re.findall(r"\b\w+\b", text)

word_freqs = {}
hashtag_freqs = {}

for inst in instances:
    print(f"\nFetching from {inst}...")
    statuses = fetch_instance_statuses(inst, START_DATE, max_pages=200, limit=40, sleep=1.0)
    print(f"  Got {len(statuses)} statuses")

    wc = Counter()
    hc = Counter()

    for s in statuses:
        html = s.get("content", "")
        text = BeautifulSoup(html, "html.parser").get_text(" ")
        words = [
            w for w in extract_words(text)
            if w not in stopwords and len(w) > 2
        ]
        wc.update(words)

        tags = s.get("tags", [])
        hc.update(t["name"].lower() for t in tags if "name" in t)

    word_freqs[inst] = wc
    hashtag_freqs[inst] = hc

TOP_N = 30

for inst in instances:
    print(f"\n==================== {inst} ====================")
    print("Top words:")
    for word, count in word_freqs[inst].most_common(TOP_N):
        print(f"{word:20} {count}")

    print("\nTop hashtags:")
    for tag, count in hashtag_freqs[inst].most_common(TOP_N):
        print(f"#{tag:19} {count}")


Fetching from sigmoid.social...
  Request failed: 429 Client Error: Too Many Requests for url: https://sigmoid.social/api/v1/timelines/public?local=true&limit=40&max_id=114784387499940989
  Giving up on this instance for now.
  Got 4080 statuses

Fetching from mastodon.ai...
  Connection error (1/3): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
  Connection error (2/3): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
  Connection error (3/3): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
  Giving up on this instance for now.
  Got 0 statuses

Fetching from ai.community...
  Request failed: 404 Client Error: Not Found for url: https://ai.community/api/v1/timelines/public?local=true&limit=40
  Giving up on this instance for now.
  Got 0 statuses

Fetching from social.ai...
  Request failed: 404 Client Error: Not Found for url: https://social.ai/api/